In [8]:
from __future__ import print_function

import logging
import numpy as np
from optparse import OptionParser
import sys, copy
from time import time

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import RidgeClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.svm import SVC

from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.extmath import density
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [21]:
'''Trains and evaluate a simple MLP
on the Reuters newswire topic classification task.
'''
from __future__ import print_function

import numpy as np
import keras
from keras.datasets import reuters
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.preprocessing.text import Tokenizer
code_list_fn = "NSCC_sample_data_170309_Codelist.dat"
all_data = open('rev_izip_kkma_all_3cols.dat',encoding='utf-8').readlines()
    
data_train_data, data_test_data = [], []
y_train, y_test = [], []
all_x, all_y = [], []
count = {}
categories =  [x for x in open(code_list_fn,'r').read().split('\n') if len(x) > 0]

for cat in categories:
    count[cat] = 0

for line in all_data:
    items = line.split('\t')
    if len(items) == 2:
        all_x.append(items[1])
        all_y.append(items[0])
        if not items[0] in count:
            count[items[0]] = 0
        count[items[0]] += 1
    else:
        print('ERROR IN READING')
data_train_data,data_test_data,y_train_from_split,y_test_from_split = train_test_split(all_x,all_y,random_state =5, train_size = 0.9)
print (len(data_train_data), len(data_test_data))
    
max_words = 1000
batch_size = 32
epochs = 5

print('Loading data...')
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_words,test_split=0.2)
(ax,ay),(bx,by) = (x_train, y_train), (x_test, y_test)                                                         
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')
print(x_train.shape)

num_classes = np.max(y_train) + 1
print(num_classes, 'classes')

print('Vectorizing sequence data...')
tokenizer = Tokenizer(num_words=max_words)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Convert class vector to binary class matrix '
      '(for use with categorical_crossentropy)')
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

print('Building model...')
model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])


60336 6704
Loading data...
8982 train sequences
2246 test sequences
(8982,)
46 classes
Vectorizing sequence data...
x_train shape: (8982, 1000)
x_test shape: (2246, 1000)
Convert class vector to binary class matrix (for use with categorical_crossentropy)
y_train shape: (8982, 46)
y_test shape: (2246, 46)
Building model...
Train on 8083 samples, validate on 899 samples
Epoch 1/5
8083/8083 [==============================] - 5s - loss: 1.4290 - acc: 0.6783 - val_loss: 1.0903 - val_acc: 0.7642
Epoch 2/5
8083/8083 [==============================] - 4s - loss: 0.7891 - acc: 0.8180 - val_loss: 0.9373 - val_acc: 0.7864
Epoch 3/5
8083/8083 [==============================] - 4s - loss: 0.5511 - acc: 0.8650 - val_loss: 0.8937 - val_acc: 0.7942
Epoch 4/5
8083/8083 [==============================] - 5s - loss: 0.4170 - acc: 0.9002 - val_loss: 0.8764 - val_acc: 0.8087
Epoch 5/5
1888/2246 [========================>.....] - ETA: 0sTest score: 0.89127845968
Test accuracy: 0.792965271594


In [25]:
ax[0]

[1,
 2,
 2,
 8,
 43,
 10,
 447,
 5,
 25,
 207,
 270,
 5,
 2,
 111,
 16,
 369,
 186,
 90,
 67,
 7,
 89,
 5,
 19,
 102,
 6,
 19,
 124,
 15,
 90,
 67,
 84,
 22,
 482,
 26,
 7,
 48,
 4,
 49,
 8,
 864,
 39,
 209,
 154,
 6,
 151,
 6,
 83,
 11,
 15,
 22,
 155,
 11,
 15,
 7,
 48,
 9,
 2,
 2,
 504,
 6,
 258,
 6,
 272,
 11,
 15,
 22,
 134,
 44,
 11,
 15,
 16,
 8,
 197,
 2,
 90,
 67,
 52,
 29,
 209,
 30,
 32,
 132,
 6,
 109,
 15,
 17,
 12]